In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import random
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
IMAGE_SIZE = (360, 363)
IMAGE_DIR = '/content/drive/MyDrive/image_data/bloodcells_dataset/'
CLASS_NAMES = ['basophil', 'eosinophil', 'erythroblast', 'ig', 'lymphocyte', 'monocyte', 'neutrophil', 'platelet']
NUM_CLASSES = len(CLASS_NAMES)

In [ ]:
# function for loading images
def load_images(image_dir, class_names, image_size):
    images = []
    labels = []

    for idx, class_name in enumerate(class_names):
        class_dir = os.path.join(image_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.jpg'):
                img = cv2.imread(os.path.join(class_dir, filename), cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, image_size)
                images.append(img)
                labels.append(idx)  # Class label is the index in the class list

    images = np.array(images)
    labels = np.array(labels)
    images = images.reshape(-1, image_size[0], image_size[1], 1)  # Adding channel dimension for grayscale

    return images, labels

In [ ]:
# Load the images and labels
images, labels = load_images(IMAGE_DIR, CLASS_NAMES, IMAGE_SIZE)

# Debugging: Check if images and labels are loaded
print(f"Total images loaded: {len(images)}")
print(f"Total labels loaded: {len(labels)}")

In [ ]:
# Check memory usage before normalization
#print(f"Memory usage before normalization: {images.nbytes / (1024 ** 2):.2f} MB")

In [ ]:

# Normalize the images and ensure they are of type float32
images = images.astype(np.float32) / 255.0


In [ ]:
# Check memory usage after normalization
#print(f"Memory usage after normalization: {images.nbytes / (1024 ** 2):.2f} MB")

In [ ]:
# Train-Test split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
# CNN Model definition
def build_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),  # Dropout to prevent overfitting
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
# Model instance
model = build_cnn((IMAGE_SIZE[0], IMAGE_SIZE[1], 1), NUM_CLASSES)

# Model summary
model.summary()

# Train the model
batch_size = 32
epochs = 20

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_test, y_test),
    epochs=epochs
)

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

In [ ]:
# display an image and its predicted class
def display_random_image_with_prediction(model, X_test, y_test, class_names):
    # Pick a random index from the test set
    random_index = random.randint(0, len(X_test) - 1)

    # Get the image and true label
    img = X_test[random_index]
    true_label = y_test[random_index]

    # Reshape the image if needed for display
    img_display = img.reshape(IMAGE_SIZE[0], IMAGE_SIZE[1])

    # Model prediction (reshape image for model if needed)
    img_for_prediction = img.reshape(1, IMAGE_SIZE[0], IMAGE_SIZE[1], 1)  # Add batch and channel dimensions
    prediction = model.predict(img_for_prediction)
    predicted_label = np.argmax(prediction)  # Get the predicted class

    # Display the image
    plt.imshow(img_display, cmap='gray')
    plt.title(f"True Label: {class_names[true_label]}, Predicted Label: {class_names[predicted_label]}")
    plt.axis('off')
    plt.show()

# Call the function to display a random image and its prediction
display_random_image_with_prediction(model, X_test, y_test, CLASS_NAMES)